# Assignment 10

## Deadline

Wednesday, December 10, 23:59.

# Assignment

In this assignment, you will practice proper data validation using several functions to calculate the distance between points on the Earth's surface.
The assignment has three tasks, which are described in the following sections.

## Notes

- Do not use the `input()` function in the final solution; your function will receive input data automatically.
- Do not insert new cells into the notebook, move existing cells, or delete them. The provided cells should be sufficient for testing your code.
- Use the **Validate** button to verify the public tests before submitting. All tests are listed below the assignment sections, but you can try validation incrementally as you go.

## Task 1

In the following cell, there is a completed function `geo_distance` that calculates the distance between two points on the Earth's surface using the [formula for calculating the orthodrome length](https://en.wikipedia.org/wiki/Haversine_formula).
The function uses the `validate_coordinates` function to validate the input data, which you need to complete.
For both parameters `coordinates1` and `coordinates2`, the following must be true:

- It is a `list` or `tuple` of length 2
- Both coordinate components have the type `float` or `int`
- The first coordinate component is [latitude](https://en.wikipedia.org/wiki/Latitude) (_zeměpisná šířka_) and must have values from -90° to 90°
- The second coordinate component is [longitude](https://en.wikipedia.org/wiki/Longitude) (_zeměpisná délka_) and must have values from -180° to 180°

If any of the above conditions are not met, the function should raise a `TypeError` or `ValueError` exception (depending on which is more appropriate in that case).

In [ ]:
import math

def validate_coordinates(coordinates):
     if isinstance(coordinates, (list, tuple)):
          if len(coordinates) == 2:
               latitude = coordinates[0]
               longitude = coordinates[1]

               if isinstance(latitude, (float, int)) and isinstance(longitude, (float, int)):
                    if (latitude >= -90 and latitude <= 90) and (longitude >= -180 and longitude <= 180):
                         return None
                    else:
                         raise ValueError
               else:
                    raise TypeError
          else:
               raise ValueError
     else:
          raise TypeError

def geo_distance(coordinates1, coordinates2):
     """ This function calculates the distance (in km) between two points
          on the Earth's surface. It assumes that the Earth is an ideal
          sphere and uses the haversine formula:
          https://en.wikipedia.org/wiki/Haversine_formula

          :param coordinates1: a pair of latitude and longitude for the first point
          :param coordinates2: a pair of latitude and longitude for the second point
          :returns: distance calculated as a float value
     """
     #* Input validation
     validate_coordinates(coordinates1)
     validate_coordinates(coordinates2)

     #* Approximate radius of earth in km
     radius = 6373 # very aproximate :D

     #* Unpack the coordinates
     lat1, lon1 = coordinates1
     lat2, lon2 = coordinates2

     #* Calculate angles in radians
     dlon = math.radians(lon2 - lon1)
     dlat = math.radians(lat2 - lat1)

     #* Apply the haversine formula
     a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
     c = math.atan2(math.sqrt(a), math.sqrt(1 - a))
     distance = 2 * radius * c

     return distance

prague = (50.0744579, 14.4142233)
brno = (49.2075088, 16.5779118)

d1 = geo_distance(prague, brno)

The distance from Prague to Brno is 192.9 km.


## Part 2

The following cell contains a partially complete function `geo_path_distance` that should calculate the length of a path formed by connecting consecutive points on the Earth's surface with the shortest line.
Complete the algorithm to calculate the distance using the `geo_distance` function from the previous part.

The `geo_path_distance` function must be usable in two ways:

1. If only one argument is provided, it must be a sequence of coordinates (i.e., a `list` or `tuple` of coordinate objects).
2. If multiple arguments are provided, those arguments directly represent the coordinate objects.

If no arguments are provided, the function should raise a `ValueError` exception.
If only one argument is provided that is not a `list` or `tuple`, the function should raise a `TypeError` exception.

Do not perform validation of the coordinates themselves in the sequence here – the `geo_distance` function will take care of that.

In [ ]:
def geo_path_distance(*coordinates):
     """ This function calculates the distance (in km) traveled by following
          points on given path. All points must be specified as pairs of
          latitude and longitude. The distance is calculated using the
          `geo_distance` function.

          :param coordinates: the coordinates can be specified in two ways:
               1. If `len(coordinates) == 1`, the first argument must be a
               sequence of points in the path.
               2. Otherwise, `coordinates` itself is the sequence of points
               in the path.
          :returns: distance calculated as a float value
     """
     #* Input adjusments and validation
     if len(coordinates) == 1:
          path = coordinates[0]
     else:
          path = coordinates

     for i in path:
          validate_coordinates(i)

     #* Corner case: geo_path_distance([["anything"]])
     if len(path) == 1:
          validate_coordinates(path)
          return 0

     #* Calculation of the distance
     distance = 0

     for i in range(len(path)):
          if i != 0:
               distance += geo_distance(path[i-1], path[i])
     return distance

prague = (50.0744579, 14.4142233)
brno = (49.2075088, 16.5779118)
ostrava = (49.8250792, 18.2373563)

# usage 1: path specified by arguments
d2 = geo_path_distance(prague, brno, ostrava)
print(f"The distance from Prague to Ostrava through Brno is {d2:.1f} km.")

# usage 2: path specified by list of points (this should be the same result as d2)
d3 = geo_path_distance([ostrava, brno, prague])
print(f"The distance from Ostrava to Prague through Brno is {d3:.1f} km.")

The distance from Prague to Ostrava through Brno is 333.7 km.
The distance from Ostrava to Prague through Brno is 333.7 km.


## Part 3

In the following cell, there is a partially prepared function `filter_invalid_coordinates` that should check all coordinates in the input sequence and divide them into 3 sets: valid coordinates, invalid coordinates causing a `TypeError` exception, and invalid coordinates causing a `ValueError` exception.

Complete the code that uses the `validate_coordinates` function for all coordinates, potentially converts the type from `list` to `tuple`, and inserts the coordinates into the correct set of coordinates.
Catch the mentioned `TypeError` and `ValueError` exceptions.
You can assume that the `filter_invalid_coordinates` function will receive an iterable object in the `coordinates` parameter – there is no need to check for cases where it receives an inappropriate object, such as a number or string.

In [ ]:
def filter_invalid_coordinates(coordinates):
     """ For each coordinate in the input sequence, this function checks if the
          coordinate is valid or invalid, and returns a tuple of three sets:
          valid coordinates, invalid coordinates that caused a `TypeError`, and
          invalid coordinates that caused a `ValueError`.
     """
     valid = set()
     type_errors = set()
     value_errors = set()

     for coords in coordinates:
          try:
               validate_coordinates(coords)
               valid.add((coords[0], coords[1]))
          except ValueError:
               value_errors.add(tuple(coords))
          except TypeError:
               if isinstance(coords, (list, tuple)):
                    type_errors.add((coords[0], coords[1]))
               else:
                    type_errors.add(coords)
     print ("final", valid, type_errors, value_errors)
     return valid, type_errors, value_errors

user_coordinates = [
     [0, 0],       # valid
     "at home",    # TypeError
     [200, 300],   # ValueError
]
filtered = filter_invalid_coordinates(user_coordinates)
print(*filtered, sep="\n")

SyntaxError: expected ':' (329115697.py, line 18)

# Tests
The following cells contain public tests that you can use for basic validation of your solution. **Click on the validate button before submitting!**

In [ ]:
# This is a read-only cell used for automatic validation - do not edit, delete, or move!
import ipytest
ipytest.autoconfig(raise_on_error=True, addopts=["-p", "no:cacheprovider"])  # type: ignore

In [ ]:
# Do not delete or edit this cell!

# nothing should happen for valid coordinates
assert validate_coordinates([0, 0]) is None
assert validate_coordinates((0, 0)) is None
assert validate_coordinates([0.0, 0.0]) is None
assert validate_coordinates([0, 0.0]) is None
assert validate_coordinates([0.0, 0]) is None
assert validate_coordinates((50.0744579, 14.4142233)) is None
assert validate_coordinates([50.0744579, 14.4142233]) is None

# end-points of the ranges are also valid
assert validate_coordinates((90, 0)) is None
assert validate_coordinates((-90, 0)) is None
assert validate_coordinates((0, 180)) is None
assert validate_coordinates((0, -180)) is None
assert validate_coordinates((90, 180)) is None
assert validate_coordinates((90, -180)) is None
assert validate_coordinates((-90, 180)) is None
assert validate_coordinates((-90, -180)) is None

In [ ]:
# Do not delete or edit this cell!
import pytest

with pytest.raises(TypeError):
    validate_coordinates("invalid coordinates")

with pytest.raises(TypeError):
    validate_coordinates({0, 0})

with pytest.raises(TypeError):
    validate_coordinates(0)

with pytest.raises(TypeError):
    geo_distance(0, 1)

with pytest.raises(TypeError):
    geo_distance("at home", "at work")

with pytest.raises(TypeError):
    geo_distance("at home", (50.0744579, 14.4142233))

with pytest.raises(TypeError):
    geo_distance((50.0744579, 14.4142233), "at work")

In [ ]:
# Do not delete or edit this cell!
import pytest

with pytest.raises(ValueError):
    validate_coordinates((1, 2, 3))

with pytest.raises(ValueError):
    validate_coordinates([1, 2, 3])

with pytest.raises(ValueError):
    validate_coordinates((1,))

with pytest.raises(ValueError):
    validate_coordinates([1,])

In [ ]:
# Do not delete or edit this cell!
import pytest

with pytest.raises(TypeError):
    validate_coordinates(("1", "2"))

with pytest.raises(TypeError):
    validate_coordinates(("1", 2))

with pytest.raises(TypeError):
    validate_coordinates((1, "2"))

with pytest.raises(ValueError):
    validate_coordinates((-90.1, 0))

with pytest.raises(ValueError):
    validate_coordinates((90.1, 0))

with pytest.raises(ValueError):
    validate_coordinates((0, -180.1))

with pytest.raises(ValueError):
    validate_coordinates((0, 180.1))

In [ ]:
# Do not delete or edit this cell!

# check that geo_path_distance accepts arguments as intended
points = [
    (50.0744579, 14.4142233),
    (49.2075088, 16.5779118),
    (49.8250792, 18.2373563),
]
assert geo_path_distance(points) > 0
assert geo_path_distance(tuple(points)) > 0
assert geo_path_distance(*points) > 0

In [ ]:
# Do not delete or edit this cell!

# check that geo_path_distance returns correct results
points = [
    (50.0744579, 14.4142233),
    (49.2075088, 16.5779118),
    (49.8250792, 18.2373563),
]
assert geo_path_distance(points[0], points[1]) == geo_distance(points[0], points[1])
assert geo_path_distance(points[0], points[2]) == geo_distance(points[0], points[2])
assert geo_path_distance(points[1], points[2]) == geo_distance(points[1], points[2])

assert geo_path_distance(points) == geo_distance(points[0], points[1]) + geo_distance(points[1], points[2])

points = [
    (50.0744579, 14.4142233),
    (49.2075088, 16.5779118),
    (49.8250792, 18.2373563),
    (50.0744579, 14.4142233),
]
assert geo_path_distance(points) == geo_distance(points[0], points[1]) + geo_distance(points[1], points[2]) + geo_distance(points[2], points[3])

In [ ]:
# Do not delete or edit this cell!

# check that geo_path_distance raises correct exception for one argument
import pytest
with pytest.raises(TypeError):
    geo_path_distance(set(points))
with pytest.raises(TypeError):
    geo_path_distance("invalid sequence of points")

# corner case that is highlighted in the implementation of the function
with pytest.raises(TypeError):
    geo_path_distance(["invalid"])
with pytest.raises(ValueError):
    geo_path_distance([["invalid"]])
with pytest.raises(TypeError):
    geo_path_distance([["invalid", "invalid"]])

In [ ]:
# Do not delete or edit this cell!

# check that filter_invalid_coordinates works for tuples
valid_points = [
    (50.0744579, 14.4142233),
    (49.2075088, 16.5779118),
    (49.8250792, 18.2373563),
]
assert filter_invalid_coordinates(valid_points) == (set(valid_points), set(), set())

In [ ]:
# Do not delete or edit this cell!

# check that filter_invalid_coordinates filters invalid coordinates as intended
points = [
    (1, 2),
    (1, 2),
    [3, 4],
    [3, 4],
    "at home",
    ["at", "work"],
    (100, 200),
    (1, 2, 3),
]
valid_points = {
    (1, 2),
    (3, 4),
}
type_errors = {
    "at home",
    ("at", "work"),
}
value_errors = {
    (100, 200),
    (1, 2, 3),
}
assert filter_invalid_coordinates(points) == (valid_points, type_errors, value_errors)

This assignment has no hidden tests 😀